In [1]:
from models.Attention import LuongAttention
from utils.data_reader import load_and_preprocess_data, load_word_embeddings
from utils.result_saver import ResultSaver
from os.path import join as pjoin
import numpy as np
import tensorflow as tf
from utils.eval import evaluate

In [2]:
tf.app.flags.DEFINE_float("learning_rate", 0.001, "Learning rate")
tf.app.flags.DEFINE_float("keep_prob", 0.8, "The probably that a node is kept after the affine transform")
tf.app.flags.DEFINE_float("max_grad_norm", 5.,
                          "The maximum grad norm during backpropagation, anything greater than max_grad_norm is truncated to be max_grad_norm")
tf.app.flags.DEFINE_integer("batch_size", 24, "Number of batches to be used per training batch")
tf.app.flags.DEFINE_integer("eval_num", 250, "Evaluate on validation set for every eval_num batches trained")
tf.app.flags.DEFINE_integer("embedding_size", 100, "Word embedding size")
tf.app.flags.DEFINE_integer("window_size", 3, "Window size for sampling during training")
tf.app.flags.DEFINE_integer("hidden_size", 100, "Hidden size of the RNNs")
tf.app.flags.DEFINE_integer("samples_used_for_evaluation", 500,
                            "Samples to be used at evaluation for every eval_num batches trained")
tf.app.flags.DEFINE_integer("num_epochs", 10, "Number of Epochs")
tf.app.flags.DEFINE_integer("max_context_length", None, "Maximum length for the context")
tf.app.flags.DEFINE_integer("max_question_length", None, "Maximum length for the question")
tf.app.flags.DEFINE_string("data_dir", "data/squad", "Data directory")
tf.app.flags.DEFINE_string("train_dir", "", "Saved training parameters directory")
tf.app.flags.DEFINE_string("retrain_embeddings", False, "Whether or not to retrain the embeddings")
tf.app.flags.DEFINE_string("share_encoder_weights", False, "Whether or not to share the encoder weights")
tf.app.flags.DEFINE_string("learning_rate_annealing", False, "Whether or not to anneal the learning rate")
tf.app.flags.DEFINE_string("ema_for_weights", False, "Whether or not to use EMA for weights")
tf.app.flags.DEFINE_string("log", True, "Whether or not to log the metrics during training")
tf.app.flags.DEFINE_string("optimizer", "adam", "The optimizer to be used ")
tf.app.flags.DEFINE_string("model", "BiDAF", "Model type")
tf.app.flags.DEFINE_string("find_best_span", True, "Whether find the span with the highest probability")

FLAGS = tf.app.flags.FLAGS

## Loading the data

In [3]:
# load the data
train, val = load_and_preprocess_data(FLAGS.data_dir)

# load the word matrix
embeddings = load_word_embeddings(FLAGS.data_dir)

# vocab map file
vocabs = []
with open(pjoin(FLAGS.data_dir, "vocab.dat")) as f:
    for line in f:
        vocabs.append(line.strip("\n"))

In [4]:
# load the result save (isn't used but is needed to initialize the model should be refactored out to be a 
# Singleton class)
result_saver = ResultSaver(FLAGS.train_dir)

## Initializing the model

In [5]:
# model = LuongAttention(result_saver, embeddings, FLAGS)
model = LuongAttention(result_saver, embeddings, FLAGS)

INFO:root:('----------', 'ENCODING ', '----------')
INFO:root:('----------', ' DECODING ', '----------')
INFO:root:answer_span_start_one_hot.get_shape() <unknown>
INFO:root:answer_span_end_one_hot.get_shape() <unknown>


## Getting a random sample

In [6]:
# Load a random sample from the validation set
n_val_samples = len(val["context"])

index = np.random.choice(np.arange(n_val_samples))

sample_data = {}
for k, v in val.items():
    sample_data[k] = v[[index]]

### The context paragraph

In [7]:
context = " ".join([word for word in sample_data["word_context"]])
context

'Beginning with the advent of beat match DJing , in which Bronx DJs ( Disc Jockeys ) including Grandmaster Flash , Afrika Bambaataa and DJ Kool Herc extended the breaks of funk records , a major new musical genre emerged that sought to isolate the percussion breaks of hit funk , disco and soul songs . As hip hop \'s popularity grew , performers began speaking ( " rapping " ) in sync with the beats , and became known as MCs or emcees . The Herculoids , made up of Herc , Coke La Rock , and DJ Clark Kent , were the earliest to gain major fame . The Bronx is referred to in hip-hop slang as " The Boogie Down Bronx " , or just " The Boogie Down " . This was hip-hop pioneer KRS-One \'s inspiration for his thought provoking group BDP , or Boogie Down Productions , which included DJ Scott La Rock . Newer hip hop artists from the Bronx include Big Pun , Lord Toriq and Peter Gunz , Camp Lo , Swizz Beatz , Drag-On , Fat Joe , Terror Squad and Corey Gunz .\n'

### The question to be answered

In [8]:
# See what the question is
question = " ".join([vocabs[word] for word in sample_data["question"][0]])
question

'Where is Big Pun from ?'

### The answer

In [9]:
answer = " ".join([word for word in sample_data["word_answer"]])
answer

'the Bronx\n'

### Predict the answer with the model

In [10]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, pjoin("README-files", "Attention-model", "LuongAttention-320"))
    start_index, end_index = model.predict_for_batch(sess, sample_data, FLAGS.find_best_span)
    pred, truth = model.get_sentences_from_indices(sample_data, start_index, end_index)

INFO:tensorflow:Restoring parameters from README-files/Attention-model/LuongAttention-320


INFO:tensorflow:Restoring parameters from README-files/Attention-model/LuongAttention-320


In [11]:
print("Prediction from model is: {}".format(" ".join(pred)))
print("Ground truth is: {}".format(" ".join(truth)))

Prediction from model is: Bronx DJs
Ground truth is: the Bronx


--------------------------------------------------------------

### See the difference between using EMA'ed weights and non-EMA weights

In [12]:
tf.reset_default_graph()
with tf.Session() as sess:
    tf.train.import_meta_graph(pjoin("README-files", "Attention-model", "LuongAttention-320.meta"))
    restore_vars = {}
    ema = tf.train.ExponentialMovingAverage(1.0)
    for var in tf.trainable_variables():
        restore_vars[ema.average_name(var)] = var
    saver = tf.train.Saver(restore_vars)
    saver.restore(sess, pjoin("README-files", "Attention-model", "LuongAttention-320"))
    start_index, end_index = model.predict_for_batch(sess, val, FLAGS.find_best_span)
    pred, truth = model.get_sentences_from_indices(val, start_index, end_index)
    res = evaluate(pred, truth)

INFO:tensorflow:Restoring parameters from README-files/Attention-model/LuongAttention-320


INFO:tensorflow:Restoring parameters from README-files/Attention-model/LuongAttention-320


NotFoundError: Key Attention/attention/Ws/ExponentialMovingAverage not found in checkpoint
	 [[Node: save/RestoreV2_63 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_1_0, save/RestoreV2_63/tensor_names, save/RestoreV2_63/shape_and_slices)]]
	 [[Node: save/RestoreV2_64/_1 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_110_save/RestoreV2_64", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'save/RestoreV2_63', defined at:
  File "/Users/kevinlu/miniconda3/envs/squad/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/kevinlu/miniconda3/envs/squad/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/kevinlu/miniconda3/envs/squad/lib/python3.6/site-packages/ipykernel_launcher.py", li

In [23]:
print("F1 score on validation set is: {}".format(res["f1"]))
print("EM on validation set is: {}".format(res["EM"]))

F1 score on validation set is: 67.57968250309993
EM on validation set is: 52.96451914098973


In [22]:
with tf.Session() as sess:
    
    saver = tf.train.import_meta_graph(pjoin("README-files", "Attention-model", "LuongAttention-1.meta"))
    restore_vars = {}
#     print('------------------------------------------------------')
#     for var in tf.global_variables():
#         print('all variables: ' + var.op.name)
#     for var in tf.trainable_variables():
#         print('normal variable: ' + var.op.name)
#     for var in tf.moving_average_variables():
#         print('ema variable: ' + var.op.name)
#     print('------------------------------------------------------')
    ema = tf.train.ExponentialMovingAverage(1.0)
    for var in tf.trainable_variables():
        restore_vars[ema.average_name(var)] = var
   
    saver = tf.train.Saver(restore_vars, name="ema_restore")
    
    saver.restore(sess, pjoin("README-files", "Attention-model", "LuongAttention-1"))
    start_index, end_index = model.answer(sess, val, FLAGS.find_best_span)
    pred, truth = model.get_sentences_from_indices(sample_data, start_index, end_index)
    f1, EM = evaluate(pred, truth)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 18)

In [ ]:
print("F1 score on validation set is: {}".format(res["f1"]))
print("EM on validation set is: {}".format(res["EM"]))

In [28]:
import tensorflow as tf
import numpy as np

    
tf.reset_default_graph()

# Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3

# Try to find values for W and b that compute y_data = W * x_data + b
# (We know that W should be 0.1 and b 0.3, but TensorFlow will
# figure that out for us.)
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0), name='W')
b = tf.Variable(tf.zeros([1]), name='b')
y = W * x_data + b

# Minimize the mean squared errors.
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
opt_op = optimizer.minimize(loss)

# Track the moving averages of all trainable variables.
ema = tf.train.ExponentialMovingAverage(decay=0.9999)
variables = tf.trainable_variables()
print(variables)
averages_op = ema.apply(tf.trainable_variables())
with tf.control_dependencies([opt_op]):
    train_op = tf.group(averages_op)

# Before starting, initialize the variables.  We will 'run' this first.
init = tf.global_variables_initializer()

saver = tf.train.Saver(tf.trainable_variables())

# Launch the graph.
sess = tf.Session()
sess.run(init)

# Fit the line.
for _ in range(201):
    sess.run(train_op)

w_reference = sess.run('W/ExponentialMovingAverage:0')
b_reference = sess.run('b/ExponentialMovingAverage:0')

saver.save(sess, os.path.join("model_ex1"))

tf.reset_default_graph()

tf.train.import_meta_graph("model_ex1.meta")
sess = tf.Session()

print('------------------------------------------------------')
for var in tf.global_variables():
    print('all variables: ' + var.op.name)
for var in tf.trainable_variables():
    print('normal variable: ' + var.op.name)
for var in tf.moving_average_variables():
    print('ema variable: ' + var.op.name)
print('------------------------------------------------------')

restore_vars = {}
ema = tf.train.ExponentialMovingAverage(1.0)
for var in tf.trainable_variables():
    print('%s: %s' % (ema.average_name(var), var.op.name))
    restore_vars[ema.average_name(var)] = var


saver = tf.train.Saver(restore_vars, name='ema_restore')

saver.restore(sess, os.path.join("model_ex1"))

w_restored = sess.run('W:0')
b_restored = sess.run('b:0')

print(w_reference)
print(w_restored)
print(b_reference)
print(b_restored)

[<tf.Variable 'W:0' shape=(1,) dtype=float32_ref>, <tf.Variable 'b:0' shape=(1,) dtype=float32_ref>]
------------------------------------------------------
all variables: W
all variables: b
all variables: W/ExponentialMovingAverage
all variables: b/ExponentialMovingAverage
normal variable: W
normal variable: b
ema variable: W
ema variable: b
------------------------------------------------------
W/ExponentialMovingAverage_1: W
b/ExponentialMovingAverage_1: b
INFO:tensorflow:Restoring parameters from model_ex1


INFO:tensorflow:Restoring parameters from model_ex1


NotFoundError: Key b/ExponentialMovingAverage_1 not found in checkpoint
	 [[Node: ema_restore/RestoreV2_1 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_ema_restore/Const_0, ema_restore/RestoreV2_1/tensor_names, ema_restore/RestoreV2_1/shape_and_slices)]]
	 [[Node: ema_restore/RestoreV2/_3 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_3_ema_restore/RestoreV2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'ema_restore/RestoreV2_1', defined at:
  File "/Users/kevinlu/miniconda3/envs/squad/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/kevinlu/miniconda3/envs/squad/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/kevinlu/miniconda3/envs/squad/lib/python3.6/site-packages/ipykernel_

In [25]:
test_restore_ema()

[<tf.Variable 'W:0' shape=(1,) dtype=float32_ref>, <tf.Variable 'b:0' shape=(1,) dtype=float32_ref>]
------------------------------------------------------
all variables: W
all variables: b
all variables: W/ExponentialMovingAverage
all variables: b/ExponentialMovingAverage
normal variable: W
normal variable: b
ema variable: W
ema variable: b
------------------------------------------------------
W/ExponentialMovingAverage: W
b/ExponentialMovingAverage: b
INFO:tensorflow:Restoring parameters from model_ex1


INFO:tensorflow:Restoring parameters from model_ex1


NotFoundError: Key b/ExponentialMovingAverage not found in checkpoint
	 [[Node: ema_restore/RestoreV2_1 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_ema_restore/Const_0, ema_restore/RestoreV2_1/tensor_names, ema_restore/RestoreV2_1/shape_and_slices)]]
	 [[Node: ema_restore/RestoreV2/_3 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_3_ema_restore/RestoreV2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'ema_restore/RestoreV2_1', defined at:
  File "/Users/kevinlu/miniconda3/envs/squad/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/kevinlu/miniconda3/envs/squad/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/kevinlu/miniconda3/envs/squad/lib/python3.6/site-packages/ipykernel_la